In [1]:
import pandas as pd



In [2]:
df_path = "/data/gc/rocksdb-index/openPMD-viewer/results/500g_iteration_10000/benchmark_result_selected_15_test.csv"

df = pd.read_csv(df_path, header=[0])

In [3]:
# print all df columns
print(df.columns)

Index(['query_seq', 'target_percentage', 'select_set', 'test_type',
       'test_name', 'index_type', 'storage', 'secondary', 'direct_block_read',
       'read_groups', 'skip_offset', 'query_index_time_elapsed',
       'remove_duplication_time_elapsed', 'sort_block_metadata_time_elapsed',
       'find_optimal_read_solution_time_elapsed',
       'generate_select_array_time_elapsed', 'get_target_data_time_elapsed',
       'get_support_data_time_elapsed', 'data_calculation_time_elapsed',
       'data_apply_select_time_elapsed',
       'apply_particle_level_select_array_time_elapsed', 'total_time_elapsed',
       'query_result_size', 'chunk_range_size', 'current_percentage',
       'iteration', 'species', 'expand_set', 'envelope', 'data_size'],
      dtype='object')


In [4]:
# deduplication and keep the last by the column [query_seq target_percentage select_set  test_type test_name index_type storage secondary direct_block_read read_groups envelope]
df = df.drop_duplicates(subset=['test_type', 'envelope'], keep='last')

In [5]:

# remove the row that 'data_size' is NaN
df = df.dropna(subset=['data_size'])



In [6]:
# sort by query_seq target_percentage test_type
df = df.sort_values(by=['target_percentage', 'envelope', 'test_type'])



In [7]:
final_df_path = "/data/gc/rocksdb-index/openPMD-viewer/results/final/large.csv"
final_df = pd.read_csv(final_df_path, header=[0])

# only keep the envelope that is in the final_df's envelope
df = df[df['envelope'].isin(final_df['envelope'])]

In [8]:
# reset index
df = df.reset_index(drop=True)

In [9]:
# keep the rows, where after grouping the df by envelope, if the group equals to 5
df = df.groupby('envelope').filter(lambda x: len(x) == 7)

In [10]:
# print the count after grouping by target_percentage and select_set, and then count the size of each group / 5
print(df.groupby(['target_percentage', 'select_set']).size() / 7)

target_percentage  select_set                  
0.01%              ('ux','uy')                     20.0
                   ('ux','uy','uz')                11.0
                   ('ux','uy','uz','x')            14.0
                   ('ux','uy','uz','x','y')        14.0
                   ('ux','uy','uz','x','y','z')    32.0
                   ('ux','uy','x','y')             18.0
                   ('ux',)                         10.0
                   ('x','y')                       20.0
                   ('x','y','z')                   14.0
                   ('x',)                          12.0
dtype: float64


In [11]:
# save to csv
df.to_csv(df_path.replace('before', 'after').replace('test', 'fin'), index=False)